In [1]:
import os,sys

import numpy as np
import pandas as pd
import math
import mud.funs as mdf

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
sys.path.insert(0,os.path.dirname(os.getcwd()))

from dc_models.models import *
from dc_utils.plots import _plot_states, _print_dims, _create_paramdf


# # Statistics libraries
# from scipy.stats import uniform, norm
# from scipy.stats import gaussian_kde as GKDE

NOT USING TEX


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 640x480 with 1 Axes>

## Mud estimation example for predator prey model

- Model: 
$$
\begin{aligned}
& \frac{d x}{d t}=\alpha x-\beta x y \\
& \frac{d y}{d t}=\delta x y-\gamma y
\end{aligned}
$$
where
- $x$ is the number of prey (for example, rabbits);
- $y$ is the number of some predator (for example, foxes);
- $\frac{d y}{d t}$ and $\frac{d x}{d t}$ represent the instantaneous growth rates of the two populations;
- $t$ represents time;
- $\alpha, \beta, \gamma, \delta$ are positive real parameters describing the interaction of the two species.

**Implementation Note/Issue:**

The given ode system has a two-dimensional state vector consisting of $x$ and $y$ and 4 separate parameters ($\lambda$'s) i.e. $\alpha, \beta, \gamma, \delta$ to estimate the mud points of.

### Generate the samples from the forward model

In [2]:

def generate_forward_runs(model, args,nruns,state_samples):
    """
    """
    for i in range(nruns):
        
        if model.__eq__('lv'):
            m = lotka_volterra(**args)
        else:
            m = None

        m._run_model()
        states, params, times = m._get_outputs()
        states_dim = states.shape

        if i == 0:
            runs = states[:,::(states_dim[1]//state_samples),:]
            pdims = params.shape
            lambdas = np.zeros([nruns,pdims[0],pdims[1]])
            lambdas[0,:,:] = params

        else:
            runs = np.append(runs,states[:,::(states_dim[1]//state_samples),:],axis=0)
            lambdas[i,:,:]=params
        
    return runs,lambdas

### Generate true parameter values to be estimated for $\alpha, \beta, \gamma, \delta$

- Note we initially set drift_windows and assim_windows = 1 to attempt a non-sequential data-consistent inversion problem before attempting sequential estimation
- Initially set the true values of all four paramters to 0.5 for simplicity
- Let *state_samples* be the output of our observation operator, which in this case will be 50
- Let *num_samples* be the number of forward runs of our model, which will be 100 for this example

In [3]:
# mud parameters
state_samples =50                     # number of samples to take from true state trajectory
num_samples = 100                      # number of samples to draw per parameter


true_args ={
    'state_init': np.array([4,2]),
    'true_model': True,
    'num_tsteps': 1000,
    'tend':200,
    'num_parameters':4,
    'lambda_true': [0.5]*4,
    'supports': [[0,1]]*4,
    'drift_windows':1,
    'assim_windows':1
    }

true_states, true_lambdas = generate_forward_runs('lv',true_args,1,state_samples)

true_states.shape, true_lambdas.shape

((1, 50, 2), (1, 1, 4))

#### Generate parameter predictions for $\alpha, \beta, \gamma, \delta$

- Note we use the same support [0,1] for the uniform distributions for each parameter for simplicity

In [4]:


predict_args ={
    'state_init': np.array([4,2]),
    'true_model': False,
    'num_tsteps': 1000,
    'tend':200,
    'num_parameters':4,
    'supports': [[0,1]]*4,
    'drift_windows':1,
    'assim_windows':1
    }


predict_states, predict_lambdas = generate_forward_runs('lv',predict_args,num_samples,state_samples)

predict_states.shape, predict_lambdas.shape

((100, 50, 2), (100, 1, 4))

### This section can be ignored for now

In [7]:
# truelv_df = _create_paramdf(true_lambdas[0],cols='pp',names=None)
# predictlv_df = _create_paramdf(predict_lambdas[0],cols='pp',names=None)

# print(f"True sampled state (qoi_true) dimensions: {true_states.shape}\n")
# print(f"True lambda value dimensions: {true_lambdas.shape} \n")
# display(truelv_df)
# print(f"Predicted sampled state (qoi) dimensions: {predict_states.shape} \n")
# print(f"Predicted lambda value dimensions: {predict_lambdas.shape} \n")
# display(predictlv_df)

True sampled state (qoi_true) dimensions: (1, 50, 2)

True lambda value dimensions: (1, 1, 4) 



,$\alpha$,$\beta$,$\delta$,$\gamma$
Drift_Window,,,,
0,0.5,0.5,0.5,0.5


Predicted sampled state (qoi) dimensions: (100, 50, 2) 

Predicted lambda value dimensions: (100, 1, 4) 



,$\alpha$,$\beta$,$\delta$,$\gamma$
Drift_Window,,,,
0,0.515975,0.107707,0.802636,0.869838


## Mud Estimation

#### Case: 1D state vector (we choose $x$ arbitrarily) and one scalar $\lambda$ parameter 
#### i.e. $x \in \mathbb{R}^{Nobs \times 1}, \quad \lambda \in \mathbb{R}$



In [28]:
qoi_true = true_states[0,:,0]
qoi = predict_states[:,:,0]
lam = predict_lambdas[:,0,0]
print(qoi_true.shape)
print(qoi.shape)
print(lam.shape)


(50,)
(100, 50)
(100,)


In [29]:
mud_problem = mdf.mud_problem(
                             domain = [[0,1]],
                             lam = lam,
                             qoi = qoi,
                             sd = np.sqrt(1.5e-3),
                             qoi_true = qoi_true,
                             num_obs = state_samples,
                                )

##### Result:  Although the estimate is crude, it is indeed possible via the mud package.

In [23]:
mud_problem.estimate()

array([0.74099233])

 #### Case: 2D state vector and one scalar $\lambda$ parameter 
### i.e. $\mathbf{x} = (x,y) \in \mathbb{R}^{Nobs \times 2}, \quad \lambda \in \mathbb{R} $

In [7]:
qoi_true2d = true_states[0,:,:]
qoi2d = predict_states
lam = predict_lambdas[:,0,0]
print(qoi_true2d.shape)
print(qoi2d.shape)
print(lam.shape)

(50, 2)
(100, 50, 2)
(100,)


##### Result: Cannot construct the map of a 2D state vector

In [8]:
mud_problem2d = mdf.mud_problem(
                             domain = [[0,1]],
                             lam = lam,
                             qoi = qoi2d,
                             sd = np.sqrt(1.5e-3),
                             qoi_true = qoi_true2d,
                             num_obs = state_samples,
                                )

ValueError: operands could not be broadcast together with shapes (50,2) (50,) 

#### Case: 1D state vector (we choose $x$ arbitrarily) and 4 dimensional $\lambda$ parameter vector
####  i.e. $x \in \mathbb{R}^{Nobs \times 1}, \quad \lambda \in \mathbb{R}^{4} $

In [11]:
qoi_true = true_states[0,:,0]
qoi = predict_states[:,:,0]
lam4d = predict_lambdas
print(qoi_true.shape)
print(qoi.shape)
print(lam4d.shape)

(50,)
(100, 50)
(100, 1, 4)


In [13]:
mud_problem4dparam = mdf.mud_problem(
                             domain = [[0,1]],
                             lam = lam4d,
                             qoi = qoi,
                             sd = np.sqrt(1.5e-3),
                             qoi_true = qoi_true,
                             num_obs = state_samples,
                                )

##### Result: Cannot construct estimate the 4d parameter vector

In [14]:
mud_problem4dparam.estimate()

ValueError: operands could not be broadcast together with shapes (100,4) (100,) 